In [1]:
#!/usr/bin/python
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
from IPython.core.debugger import set_trace

In [51]:
# IMPORT PACKAGES
from __future__ import print_function
import os
import numpy as np
import logging
import json
from pathlib import Path
from tqdm import tqdm
from pathlib import Path
# from skimage import measure, io
# from shapely.geometry import Polygon, MultiPolygon
import PIL
from PIL import ImageDraw
from pycocotools import mask
from pycocotools.cocostuffhelper import cocoSegmentationToPng
from pycocotools.coco import COCO
import skimage.io

import matplotlib.pyplot as plt
# 
from utils import get_varargin, timeit
import logger_utils

ModuleNotFoundError: No module named 'pycocotools.cocostuffhelper'

In [3]:
# SETUP
logger_utils.logging_setup()
#	Working Directories
current_dir = os.getcwd()
user_dir = os.path.expanduser('~')
# script_dir = os.path.dirname(os.path.abspath(__file__))

In [50]:
class coco_dataset(object):
# Initialize Class
    def __init__(self, **kwargs):
        json_filepath = get_varargin(kwargs, 'json_filepath', 'untitiles.json')
        img_dir = get_varargin(kwargs,'img_dir', os.path.dirname(json_filepath))
        self.json_filepath = json_filepath
        self.img_dir = img_dir
        self.load_json()
# =================================================================================================================
    @property
    def json_filepath(self):
        return self._json_filepath
    @json_filepath.setter
    def json_filepath(self, filepath):
        self._json_filepath = filepath
    @property
    def img_dir(self):
        return self._img_dir
    @img_dir.setter
    def img_dir(self, input_dir):
        self._img_dir = input_dir
# =================================================================================================================
# METHODS
    def load_json(self, **kwargs):
        """
        Load .json file and create json object
        """
        if not os.path.isfile(self.json_filepath):
            logging.error('{} is not exist'.format(self.json_filepath))
            raise FileNotFoundError('{} is not exist'.format(self.json_filepath))
        else:
            with open(self.json_filepath) as fid:
                logging.info('Open file: {}'.format(self.json_filepath))
                self.json_obj = json.load(fid)
    
    def get_keys(self, **kwargs):
        key_list = list(self.json_obj.keys())
        display_opt = get_varargin(kwargs, 'display', False)
        self.json_key_list = key_list
        if display_opt is True:
            print(key_list) 
    
    def get_json_data(self, **kwargs):
        """
        Get key value from json object
        Options:
            display: Print key values info. Default: True
            keys: List of key to get value. Default: All key
        """
        display_opt = get_varargin(kwargs, 'display', True)
        self.get_keys()
        input_keys = get_varargin(kwargs, 'keys', self.json_key_list)
        for key in input_keys:
            if key in self.json_key_list:
                strcmd = "self.{} = self.json_obj.get('{}')".format(key,key)
                exec(strcmd)
                if display_opt is True:
                    strcmd = "print(self.{})".format(key)
                    exec(strcmd)
            else:
                logging.info('Key not exist in coco json file: {}'.format(key))
        
    def get_info(self, **kwargs):
        """
        Get values from 'info' key from json file object
        """
        display_opt = get_varargin(kwargs, 'display', True)
        self.get_json_data(keys = ['info'], display = display_opt)
        return self.info
    
    def get_licenses(self, **kwargs):
        """
        Get license info from coco .json file
        """        
        display_opt = get_varargin(kwargs, 'display', True)
        self.get_json_data(keys = ['licenses'], display = display_opt)
        return self.licenses
    
    def get_categories(self, **kwargs):
        """
        Get categories information from .json file
        """
        display_opt = get_varargin(kwargs, 'display', True)
        self.get_json_data(keys = ['categories'], display = display_opt)
        print('Number of Categories: {}'.format(len(self.categories)))
        return self.categories

    def get_images(self, **kwargs):
        """
        Get images information from .json file
        """
        display_opt = get_varargin(kwargs, 'display', False)
        verbose = get_varargin(kwargs, 'verbose', False)
        self.get_json_data(keys = ['images'], display = display_opt)
        if verbose is True:
            print('Number of Images: {}'.format(len(self.images)))
            print('Example Image Info:')
            print('\t{}'.format(self.images[0]))
        return self.images
    
    def get_annotations(self, **kwargs):
        """
        Get annotations information from .json file
        """
        display_opt = get_varargin(kwargs, 'display', False)
        verbose = get_varargin(kwargs, 'verbose', False)
        self.get_json_data(keys = ['annotations'], display = display_opt)
        if verbose is True:
            print('Number of Annotations: {}'.format(len(self.annotations)))
            print('Example Annotations Info:')
            print('\t{}'.format(self.annotations[0]))
        return self.annotations
    
    def get_image_annotations(self, **kwargs):
        """
        """
        img_id = get_varargin(kwargs, 'id', 0)
        display_opt = get_varargin(kwargs, 'display', False)
        coco_anno = self.get_annotations()
        for anno in coco_anno:
            if img_id == anno['id']:
                if display_opt is True:
                    print(anno)
                return anno
    
    def display_image(self, **kwargs):
        """
        """
        
        img_id = get_varargin(kwargs, 'id', 0)
        coco_img = self.get_image_info(id = img_id)
        image_path = os.path.join(self.img_dir, coco_img['file_name'])
        img = PIL.Image.open(image_path)
        
        img_anno = self.get_image_annotations(id = img_id)
        segment = img_anno['segmentation']
        x = segment[::2]
        y = segment[1::2]
        xy = [(x[i], y[i]) for i in range(len(x))]
        print(xy)
        img2 = img.copy()
        draw = ImageDraw.Draw(img2)
        draw.polygon(xy, fill = "wheat")

        img3 = PIL.Image.blend(img, img2, 0.5)
        img3.show()
#         plt.imshow(img3)
            
    def get_image_info(self, **kwargs):
        """
        Get image coco information from image id
        """
        img_id = get_varargin(kwargs, 'id', 0)
        coco_img = self.get_images()
        for img in coco_img:
            if img_id == img['id']:
                return img
    
    
# =================================================================================================================
json_filepath = Path(user_dir) / 'serviceBot/datasets/via_sample/via_export_coco.json'
train_imgpath = Path(user_dir) / 'serviceBot/datasets/via_sample/train'
coco_obj = coco_dataset(json_filepath = json_filepath, img_dir = train_imgpath)
coco_obj.get_info()
coco_obj.get_image_info(id = 0)
coco_obj.display_image(id = 0)
coco_obj.get_image_annotations(id = 0)

20191031-01:16 | <ipython-input-50-d8937193d41e> - INFO - Open file: /home/phatluu/serviceBot/datasets/via_sample/via_export_coco.json
{'year': 2019, 'version': '1', 'description': 'Exported using VGG Image Annotator (http://www.robots.ox.ac.uk/~vgg/software/via/)', 'contributor': '', 'url': 'http://www.robots.ox.ac.uk/~vgg/software/via/', 'date_created': 'Wed Oct 30 2019 14:00:13 GMT-0500 (Central Daylight Time)'}
[(116, 157), (94, 195), (176, 264), (343, 273), (383, 261), (385, 234), (369, 222), (406, 216), (398, 155), (364, 124), (310, 135), (297, 170), (304, 188), (244, 170), (158, 175)]


{'id': 0,
 'image_id': '0',
 'segmentation': [116,
  157,
  94,
  195,
  176,
  264,
  343,
  273,
  383,
  261,
  385,
  234,
  369,
  222,
  406,
  216,
  398,
  155,
  364,
  124,
  310,
  135,
  297,
  170,
  304,
  188,
  244,
  170,
  158,
  175],
 'area': 46488,
 'bbox': [94, 124, 312, 149],
 'iscrowd': 0}

In [30]:
anno = coco_obj.get_image_annotations(id = 0, display = False)
segment = anno['segmentation']
print(segment)

[116, 157, 94, 195, 176, 264, 343, 273, 383, 261, 385, 234, 369, 222, 406, 216, 398, 155, 364, 124, 310, 135, 297, 170, 304, 188, 244, 170, 158, 175]


In [31]:
type(segment)

list

In [33]:
segment[::2]

[116, 94, 176, 343, 383, 385, 369, 406, 398, 364, 310, 297, 304, 244, 158]

In [43]:
segment[1::2]

[157, 195, 264, 273, 261, 234, 222, 216, 155, 124, 135, 170, 188, 170, 175]